In [ ]:

#Exploring distance of mitochondria to chloroplasts and Cell Walls

In [ ]:
from sfepy.discrete.fem import Mesh, FEDomain, Field
import os
from stl import mesh
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import pyvista as pv
import numpy as np
import networkx as nx
from scipy.spatial import KDTree
import seaborn as sns
import vtk
import time
from IPython.display import display
import itkwidgets



In [ ]:
#Ideally we would read in images and convert them to meshes (3D objects) in Python-
#Im just doing this off an STL - the results (currently the units are wrong so we can only do relative) should be similar
tstart = time.time()

In [ ]:
os.chdir("F:/FEMPython/")

t0 = time.time()
chl= pv.read("F:/CHICKPEA MIT PROJECT/D2Cell1-This is HT/D2Cell1STLS(20nmx50nm)/HT-CHL.stl")
#reduce the mesh quality for exploring (then change it for HQ results)
target_reduction = 0.0
chl=chl.decimate(target_reduction)
#######################################

mit= pv.read("F:/CHICKPEA MIT PROJECT/D2Cell1-This is HT/D2Cell1STLS(20nmx50nm)/HT-MIT.stl")
mit=mit.decimate(target_reduction)
#mit=mitraw.decimate(target_reduction)
mit2=mit #wastes memory and is slow
mit3=mit

air=pv.read("F:/CHICKPEA MIT PROJECT/D2Cell1-This is HT/D2Cell1STLS(20nmx50nm)/HT-AIR.stl")
air=air.decimate(target_reduction)
adjcells=pv.read("F:/CHICKPEA MIT PROJECT/D2Cell1-This is HT/D2Cell1STLS(20nmx50nm)/HT-ADJRESHAPED.stl")
adjcells=adjcells.decimate(target_reduction)
cwproxy=adjcells+air
t1 = time.time() - t0

print('STL Load in Time: '+str(np.round(t1, 1))+' s')

In [ ]:
#Now its best to visulaise it all

print("Chickpea Cell (HT)")



p = pv.Plotter()
p.title=("chickpea cell")
p.add_mesh(chl, color="green")
p.add_mesh(mit, color="red")
p.add_mesh(air, color="blue", opacity=0.1)
p.add_mesh(adjcells, color="white", opacity=1)
p.add_bounding_box()
p.set_background("white", top="white")
p.show()



In [ ]:
def plot_scene_1():
    from IPython.display import display
    pv.set_plot_theme("document")
    plotter = pv.Plotter()
    plotter.add_mesh(mit, color="red")
    plotter.add_mesh(chl, color="green", opacity=1)
    plotter.add_mesh(air, color="blue", opacity=0.6)
    plotter.add_mesh(adjcells, color="white", opacity=0.6)
    disp = plotter.show(use_panel=True, auto_close=False)
    display(disp)
plot_scene_1()

In [ ]:
#Now we want to get the distances from each mitochdnria to the nearest chloroplast 
#If a mitochondria is closer there is more change for CO2 reffixation 

In [ ]:
t0 = time.time()
tree = KDTree(chl.points)
d, idx = tree.query(mit.points )
mit["Distance (um)"] = d/1000
t1 = time.time() - t0
print('Ktree Run Time: '+str(np.round(t1, 1))+' s')
print ('mean distance from mit to chl: '+str(np.mean(d)/1000)+ ' um')

In [ ]:
print("saving each points differences in nm")
np.savetxt("HTKDtreeMit-CHL.csv", d, delimiter=",")

In [ ]:
#create a unique dataframe which has the mit surffcaes and there distances from chlroplasts 
chldistances=mit

In [ ]:
print("The distance between the surface of mitchondria and chlroplasts")

sargs = dict(
    title_font_size=20,
    label_font_size=15,
    shadow=True,
    n_labels=5,
    italic=True,
    fmt="%.1f",
    font_family="arial",height=0.5, 
    vertical=True, 
    position_x=0.3,
    position_y=0.8
)

boring_cmap = plt.cm.get_cmap("jet", 50)

pv.set_plot_theme("document")
p = pv.Plotter()
p.set_background("white")
p.add_mesh(chldistances, scalars="Distance (um)",scalar_bar_args=sargs, cmap=boring_cmap )
p.add_mesh(chl, color="green", opacity=0.5)
p.show()


In [ ]:
#plotter = pv.PlotterITK()
#plotter.add_mesh(chl, color="green")
#plotter.add_mesh(mit, color="red")
#plotter.add_mesh(air, color="blue", opacity=0.1)
#plotter.add_mesh(adjcells, color="white", opacity=1)
#plotter.show(True)

In [ ]:
print("The distance between the surface of mitchondria and chlroplasts")
print("(chloroplasts not shown)")
pv.set_plot_theme("document")
p = pv.Plotter()
p.add_mesh(chldistances, scalars="Distance (um)",scalar_bar_args=sargs, cmap=boring_cmap )
p.show()


In [ ]:
def plot_scene_1():
    from IPython.display import display
    pv.set_plot_theme("document")
    plotter = pv.Plotter()
    plotter.add_mesh(chldistances, scalars="Distance (um)",scalar_bar_args=sargs, cmap=boring_cmap )
    plotter.add_mesh(chl, color="green", opacity=0.6)
    disp = plotter.show(use_panel=True, auto_close=False)
    display(disp)
plot_scene_1()

In [ ]:
t0 = time.time()
tree2 = KDTree(cwproxy.points)
d2, idx = tree2.query(mit2.points )
mit2["Distance (um)"] = d2/1000
t1 = time.time() - t0
print('Ktree Run Time: '+str(np.round(t1, 1))+' s')
print ('mean distance from mit to chl: '+str(np.mean(d2)/1000)+ ' um')



In [ ]:
print("saving each points differences in nm")
np.savetxt("HTKDtreeMit-cellwall.csv", d2, delimiter=",")

In [ ]:
#create a unique dataframe which has the mit surffcaes and there distances from cellwall 
cwdistances=mit2

In [ ]:
#So on average mitochondria are closer to the cell wall.(stress this is just one celll)
print("the difference between distances")
np.mean(d2 - d)/1000

In [ ]:
print("The distance between the surface of mitchondria and cell walls")
pv.set_plot_theme("document")
p = pv.Plotter()
p.add_mesh(cwdistances, scalars="Distance (um)",scalar_bar_args=sargs, cmap=boring_cmap )
p.add_mesh(cwproxy, color=True, opacity=0.1)
p.show()


In [ ]:
print("The distance between the surface of mitchondria and cellwalls")
print("The Cell Wall is removed")
pv.set_plot_theme("document")
p = pv.Plotter()
p.add_mesh(cwdistances, scalars="Distance (um)",scalar_bar_args=sargs, cmap=boring_cmap)
p.show()



In [ ]:

def plot_scene_1():
    from IPython.display import display
    pv.set_plot_theme("document")
    plotter = pv.Plotter()
    plotter.add_mesh(cwdistances, scalars="Distance (um)",scalar_bar_args=sargs, cmap=boring_cmap )
    plotter.add_mesh(cwproxy, color=True, opacity=0.1)
    disp = plotter.show(use_panel=True, auto_close=False)
    display(disp)
plot_scene_1()

In [ ]:
#Now for every point on the mitochdnria we have 
#1) the distance to the nearest chorloplast 
#2) the distance to the nearest cell wall

# so we can calculate the difference for each point (cell wall distance - chlroplast distance)
# a positive number means the chlroplast is closer
# a negative number means the cell wall is closer. 

In [ ]:
#Create a new object that has the difference instead of the actual distance
cwdif=mit3
dif=d2-d
cwdif["Distance (um)"]=dif/1000

In [ ]:
print("The difference between the two mesurments (cell wall distance - chl distance)")
print("darker colours (negative) are where mironcdhria are closer to the cell")
print("warmer colours (positive) are where mironcdhria are closer to the chlroplast")
pv.set_plot_theme("document")
p = pv.Plotter()
p.add_mesh(cwdif, scalars="Distance (um)",scalar_bar_args=sargs, cmap=boring_cmap )
p.add_mesh(cwproxy, color="white", opacity=0.3)
p.add_mesh(chl, color="green", opacity=0.3)
#p.add_bounding_box()
#p.set_background("white", top="white")
p.show()

In [ ]:
print("The difference between the two mesurments (cell wall distance - chl distance)")
print("cell wall and chlroplast removed")
pv.set_plot_theme("document")
p = pv.Plotter()
p.add_mesh(cwdif, scalars="Distance (um)",scalar_bar_args=sargs, cmap=boring_cmap )
p.show()

In [ ]:
print("The difference between the two mesurments (cell wall distance - chl distance)")
print("This is Binary= The warmer colours (where mit are closer to chl turn red") 
print("and the cooler colours (wheremit are closer to cell wall turn yellow")
      
scalars = np.empty(cwdif.n_points)
scalars[cwdif['Distance (um)'] < 0] = 4  # red
scalars[cwdif['Distance (um)'] > 0] = 2  # yellow
pv.set_plot_theme("document")
p = pv.Plotter()
p.add_mesh(cwdif, scalars=scalars, cmap=['red', 'yellow'])
p.add_mesh(cwproxy, color="grey", opacity=0.1)
p.add_mesh(chl, color="green", opacity=1)

#p.add_bounding_box()
#p.set_background("white", top="white")
p.show()


In [ ]:

plt.hist(dif, density=True, bins=300)  # This is difference 
sns.kdeplot(data=dif)

In [ ]:
plt.hist(d, density=True, bins=300) # mit to chl distance
sns.kdeplot(data=d)

In [ ]:
plt.hist(d2, density=True, bins=300) # mit to cellwall distance
sns.kdeplot(data=d2)

In [ ]:
plt.hist(dif, density=True, bins=300)  # `density=False` would make counts
sns.kdeplot(data=dif)
plt.hist(d, density=True, bins=300)  # `density=False` would make counts
sns.kdeplot(data=d)
plt.hist(d2, density=True, bins=300)  # `density=False` would make counts
sns.kdeplot(data=d2)

In [ ]:
############################################################################
############MCHL DISTANCE TO IAS ################################

In [ ]:
t0 = time.time()
chl1=chl
tree3 = KDTree(air.points)
d3, idx = tree3.query(chl1.points )
chl1["Distance (um)"] = d3/1000
t1 = time.time() - t0
print('Ktree Run Time: '+str(np.round(t1, 1))+' s')
print ('mean distance from chl to air: '+str(np.mean(d3)/1000)+ ' um')

In [ ]:
print("saving each points differences in nm")
np.savetxt("HTKDtreechl-air.csv", d, delimiter=",")

In [ ]:
sc=chl1

In [ ]:
print("The distance between the surface of chl and air")
pv.set_plot_theme("document")
p = pv.Plotter()
p.add_mesh(sc, scalars="Distance (um)",scalar_bar_args=sargs, cmap=boring_cmap )
#p.add_mesh(air, color="grey", opacity=0.7)
p.show()


In [ ]:
print("This is Binary= The warmer colours (where chls are closer to IAS turn red") 
print("and the cooler colours (where chl are closer to cell wall turn yellow")
pv.set_plot_theme("document")
p = pv.Plotter()
scalars = np.empty(sc.n_points)
scalars[sc['Distance (um)'] < 0.75] = 4  # red
scalars[sc['Distance (um)'] > 0.75] = 2  # yellow
pv.set_plot_theme("document")
p = pv.Plotter()
p.add_mesh(sc, scalars=scalars, cmap=['red', 'yellow'])
#p.add_mesh(cwproxy, color="grey", opacity=0.1)
#p.add_mesh(chl, color="green", opacity=1)
p.show()


In [ ]:
def plot_scene_1():
    from IPython.display import display
    pv.set_plot_theme("document")
    plotter = pv.Plotter()
    plotter.add_mesh(sc, scalars=scalars, cmap=['red', 'yellow'])
    plotter.add_mesh(air, color="blue", opacity=0.5)
    disp = plotter.show(use_panel=True, auto_close=False)
    display(disp)
plot_scene_1()

In [ ]:
t1 = time.time() - tstart
print('Total Run Time: '+str(np.round(t1, 1))+' s')

In [ ]:
#Lets get some last information 
mit



In [ ]:
chl